In [1]:
from helpers import get_model
from data import import_data
import torch


root_dir = '../../results/'
data_dir = '../../data'

parameters = {
                #'dataset': "MNIST",
                'dataset': "CIFAR10", 


                #'model_name':'LeNet',
                #'model_name':'Resnet18',

                #'loss_function': 'Crossentropy',
                #'loss_function':'Evidential_LOG',
                #'loss_function':'Evidential_DIGAMMA',


                #'model_name':'LeNet_DUQ',
                'model_name':'ResNet_DUQ',
                'loss_function': 'DUQ',
                }


model_path = str(root_dir)+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model.pth'
model = get_model(parameters['model_name'],num_classes=10,weights=None)
model.load_state_dict(torch.load(model_path))  

dataloader, class_names = import_data(parameters['dataset'], data_dir, 128, 128)
dataiter = iter(dataloader['train'])
images, labels = next(dataiter)
    
torch.onnx.export(
    model,
    images,
    root_dir+"ONNX/"+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model.onnx',
    opset_version=12,
    do_constant_folding=True,
)

import onnx
from onnxsim import simplify

onnx_model = onnx.load(root_dir+"ONNX/"+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model.onnx')
model_simpified, check = simplify(onnx_model)
onnx.save(model_simpified, root_dir+"ONNX/"+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model_sim.onnx')

import blobconverter

blobconverter.from_onnx(
    model=root_dir+"ONNX/"+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model_sim.onnx',
    output_dir=root_dir+"ONNX/"+str(parameters['loss_function'])+'_'+str(parameters['model_name'])+'_model.blob',
    data_type="FP16",
    shaves=6,
    use_cache=False,
    optimizer_params=[]
)

Files already downloaded and verified
Files already downloaded and verified
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

[=====================================             ]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

